In [102]:
import pandas as pd
import numpy as np
import os
import gc
import time
import re
import ast
import country_converter as coco
import time

import logging
coco_logger = logging.getLogger('country_converter.country_converter')
coco_logger.setLevel(logging.ERROR)

In [2]:
DF_input = input().strip()

  C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\


In [3]:
entities_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\allCountries_clean.csv


In [52]:
institutions_database_input = input().strip()

 C:\Users\svalb\OneDrive\Escritorio\Data_40_years_cancer_studies\resources\ScimagoIR 2025 - Overall Rank.csv


In [4]:
df=pd.read_csv(DF_input+"parsedX_100000.csv")

In [5]:
entities_database = pd.read_csv(entities_database_input)

In [54]:
institutions_database = pd.read_csv(institutions_database_input, sep=";")

In [55]:
institutions_database

,Global Rank,Institution,Country,Sector,Best Country Quartile
0,1,Chinese Academy of Sciences *,CHN,Government,1
1,2,Ministry of Education of the People's Republic...,CHN,Government,1
2,3,Centre National de la Recherche Scientifique *,FRA,Government,1
3,4,Harvard University *,USA,Universities,1
4,5,University of Chinese Academy of Sciences *,CHN,Universities,1
...,...,...,...,...,...
9751,9752,Iranian Research Institute of Plant Protection,IRN,Government,4
9752,9753,Al-Turath University College,IRQ,Universities,4
9753,9754,Ecole Normale Superieure de Kouba Bachir el Ib...,DZA,Universities,4
9754,9755,Semashko National Research Institute of Public...,RUS,Health,4


In [57]:
# Remove " *" from institutions when present
institutions_database["Institution"] = institutions_database["Institution"].apply(lambda x: x[:-2] if x[-2:] == " *" else x)

In [6]:
entities_database.head()

,name,asciiname,alternatenames,country code
0,Xixerella,Xixerella,NaN,AD
1,Vila,Vila,"Casas Vila,Vila",AD
2,Tossalet i Vinyals1,Tossalet i Vinyals1,"Tossalet i Vin'jal's,Tossalet i Vinyals,Тоссал...",AD
3,Sornàs,Sornas,"Sornas,Sornàs",AD
4,Soldeu,Soldeu,"Sol'deu,Soldeu,surudeu,swldw,Сольдеу,סולדאו,سو...",AD


In [105]:
institutions_database.head()

,Global Rank,Institution,Country,Sector,Best Country Quartile
0,1,Chinese Academy of Sciences,CHN,Government,1
1,2,Ministry of Education of the People's Republic...,CHN,Government,1
2,3,Centre National de la Recherche Scientifique,FRA,Government,1
3,4,Harvard University,USA,Universities,1
4,5,University of Chinese Academy of Sciences,CHN,Universities,1


In [115]:
df.loc[df["PMID"]==1279132]["Authors"].values

array(["[{'Name': 'G Karp', 'Affiliation': ['Department of Medicine, UMDNJ-Robert Wood Johnson Medical School.']}, {'Name': 'K Nahum', 'Affiliation': []}]"],
      dtype=object)

In [12]:
cc = coco.CountryConverter()

# Define a dictionary for US states
us_states_to_country = {
    "Alabama": "United States", "Alaska": "United States", "Arizona": "United States",
    "Arkansas": "United States", "California": "United States", "Colorado": "United States",
    "Connecticut": "United States", "Delaware": "United States", "Florida": "United States",
    "Georgia": "United States", "Hawaii": "United States", "Idaho": "United States",
    "Illinois": "United States", "Indiana": "United States", "Iowa": "United States",
    "Kansas": "United States", "Kentucky": "United States", "Louisiana": "United States",
    "Maine": "United States", "Maryland": "United States", "Massachusetts": "United States",
    "Michigan": "United States", "Minnesota": "United States", "Mississippi": "United States",
    "Missouri": "United States", "Montana": "United States", "Nebraska": "United States",
    "Nevada": "United States", "New Hampshire": "United States", "New Jersey": "United States",
    "New Mexico": "United States", "New York": "United States", "North Carolina": "United States",
    "North Dakota": "United States", "Ohio": "United States", "Oklahoma": "United States",
    "Oregon": "United States", "Pennsylvania": "United States", "Rhode Island": "United States",
    "South Carolina": "United States", "South Dakota": "United States", "Tennessee": "United States",
    "Texas": "United States", "Utah": "United States", "Vermont": "United States",
    "Virginia": "United States", "Washington": "United States", "West Virginia": "United States",
    "Wisconsin": "United States", "Wyoming": "United States",
    "DC": "United States", "District of Columbia": "United States", 
    "CA": "United States", "NY": "United States", "TX": "United States"
}

In [103]:
def extract_country_with_states(cell_content):
    if pd.isna(cell_content) or cell_content is None:
        return None

    elif isinstance(ast.literal_eval(cell_content), list):
        parsed_content = ast.literal_eval(cell_content)
        countries = []
        for el in parsed_content:
            if isinstance(el, dict):
                text = el["text"]
                label = el["label"]

                if text in list(us_states_to_country.keys()):
                    text = us_states_to_country[text]
                
                if label in ["LOC", "GPE"]:
                    standard_names = cc.convert(names = text, to = 'name_short')
                    if standard_names != "not found":
                        countries.append(standard_names)
            else:
                pass
            
        if len(countries) == 1:
            return countries[0]
            
        elif len(countries) == 0:
            string_parts = [el["text"] for el in parsed_content if isinstance(el, dict) and "text" in el]
            string = ", ".join(string_parts) + ", " if string_parts else ""
            for el in parsed_content:
                entity_name = el["text"]
                if len(institutions_database[institutions_database["Institution"]==entity_name]) == 1:
                    return cc.convert(institutions_database[institutions_database["Institution"]==entity_name]["Country"].values[0], to="name_short")
                
            for el in parsed_content[::-1]:
                entity_name = el["text"]
                df_entities = entities_database[entities_database["name"]==entity_name]
                if len(df_entities) == 1:
                    return cc.convert(df_entities["country code"].iloc[0], to="name_short")
            
                elif len(df_entities) > 1:
                    possible_countries = df_entities["country code"].values
                    for country in possible_countries:
                        if type(country)==str:
                            country_short_name = cc.convert(country, to="name_short")
                            if country_short_name != "not found" and re.search(r'\b' + re.escape(country_short_name) + r'\b', string):
                                return country_short_name
                else:
                    return None
                        
        else:
            count_country = {}
            for el in countries:
                if el in list(count_country.keys()):
                    count_country[el] += 1
                else:
                    count_country[el] = 1

            top_country = list(count_country.keys())[list(count_country.values()).index(max(count_country.values()))]
            return top_country

    else:
        return None

start = time.time()
df_2["Country"] = df_2["NER_lastAuthor"].apply(extract_country_with_states)

print(time.time()-start)

12.399199485778809


In [8]:
df_2 = df.copy(deep=True)

In [196]:
df_2 = df_2.iloc[0:100]